#Excercise #5

Use the *PanelObject* to derive an object with a three button user interface, with the buttons linked to display of the following functions in the output region $y=cos(x), y = log(x)$ and $y= e^x$. The following description provides a brief recipe for deriving a new object:

* Derive a new class from the panel object.  The new class will inherit the properties and functions of the panel object, which can be overridden. In the following cell, the first line specifies that *MyClass* is derived from *PanelObject* class. The second line is the constructor for the new class. The third line specifies the *title* for the panel and the fourth line calls the constructor for the *PanelObject*, which will set up the control panel widget (*cp*) which contains the *objLabel*, *inpUSR* and *objOUT* attributes, which are graphical elements that contain the top label, user interface and output display of the panel.  



In [0]:
class MyClass(PanelObject):
  def __init__(self,*args,**kwargs):
    self.title = 'My Title'
    PanelObject.__init__(self,*args, **kwargs)

NameError: ignored

* The second step is to write the *getCP()* function, where you will create the user interaction widgets, link them to appropriate callbacks and add them to the *inpUSR* widget.  If needed, you can also modify the title at this stage by setting the *title* attribute and then calling *setLabel()* function. 

In [0]:
 def getCP(self):
   #another way to change title
   self.title='My title'
   self.setLabel()    
   #add widgets and callbacks
   self.inpUSR.children+= (self.myWidget,layout={'overflow':'visible'}),)
   self.myWidget.on_click(self.myCallback)
   return self.cp


* The third step is to write appropriate callbacks that will do plotting and other analysis depending upon button press, item selection etc. The main thing to remember here is to send output to *out_cp* and to clear the output area when needed. Also, you need to include *plt.ioff()* when using *matplotlib*.

In [0]:
def myCallback(self):
  plt.ioff()
  with self.out_cp:
    self.out_cp.clear_output()
    print('some stuff')
    ax.contourf(...)


In [1]:
!apt-get update
!apt-get install -qq  libgdal-dev libproj-dev 
!pip uninstall -y shapely
!pip install shapely cartopy --no-binary shapely --no-binary cartopy
!pip install Pydap
!pip install netCDF4
!pip install py-openaq
!pip install tropycal

Hit:1 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic/main Sources [1,814

In [0]:
from ipywidgets import HBox, VBox, Button,Layout, HTML,Output,GridspecLayout,Dropdown,GridBox,Textarea,Text,Password
class PanelObject:
   def __init__(self,*args,**kwargs):
    
    if hasattr(self,'title') == False:
      self.title='Panel Object'
    
    self.topLabel = HTML(disabled=True,layout=Layout(width='auto',grid_area='title'))
    
    self.setLabel()
    
    self.rule = HTML(disabled=True,layout=Layout(width='auto',grid_area='rule',flex_flow='column'))
    
    self.rule.value = ( '<div >'+'<hr></div>')

    self.spacer = HTML(disabled=True,layout=Layout(width='auto',grid_area='rule',flex_flow='column'))

    self.out_cp = Output(layout=Layout(width='auto', grid_area='output',overflow='visible',flex_flow='column'))

    self.inpUSR = HBox([],layout={'width':'auto','overflow':'visible','height':'100px'})

    self.objOUT = HBox([VBox([self.out_cp])]
                       ,layout={'width':'auto','overflow':'visible','grid_area':'output'})
    self.objLabel= HBox([self.topLabel],layout={'overflow':'visible','width':'auto','justify_content':'center','flex_flow':'column','align_item':'stretch'})
    
    self.cp  = VBox([self.objLabel,self.inpUSR,self.rule,self.spacer,self.objOUT],layout={'width':'auto','overflow':'visible','flex_flow':'column'})

    self.pwdDict= {}

   def setLabel(self):
    self.topLabel.value = ( '<div style="background-color: #C0C0C0 ;'+ 
                           'padding: 1em 0em 0em 0em; border-bottom-style:solid;border-width:1px">'+
                           '<b><center><h3>'+self.title+'</h3></center></b></div>')

In [3]:
from IPython.display import display
from ipywidgets import HBox, VBox, Button,Layout, HTML,Output,GridspecLayout,Dropdown,GridBox,Textarea,Text,Password
import openaq
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


class OpenAQGui(PanelObject):
  def __init__(self,*args,**kwargs):
    self.title = 'OpenAQ GUI'
    PanelObject.__init__(self,*args, **kwargs)

    # initialize openaq api

    self.api        = openaq.OpenAQ()

    self.createGuiElemenents()

  def getCP(self):
    return self.cp

  def createGuiElemenents(self):

    # setup selection widgets

    self.countrySW = Dropdown(options=[],
                              value=None,
                              description='Country:',
                              layout=Layout(width='250px', grid_area='country'),
                              disabled=False
                             )
    
    self.citySW    = Dropdown(options=[],
                              value=None,
                              description='City:',
                              layout=Layout(width='250px', grid_area='city'),
                              disabled=False
                             )
    
    self.locationSW = Dropdown(options=[],
                               value=None,
                               description='Location:',
                               layout=Layout(width='250px', grid_area='location'),
                               disabled=False
                              )
    self.inpUSR.children+= (VBox([self.countrySW,self.citySW,self.locationSW],layout={'overflow':'visible'}),)
    
    # get the list of countries. Set the default country selection 
    # to be the first country on the list and extract the corresponding 
    # country code

    self.countries  = self.api.countries(df=True,limit=1000)
    self.country    = self.countries.iloc[0]['name']
    self.ccode    = self.countries.iloc[0]['code']
    
    self.countrySW.options = list(self.countries['name'])

    self.countrySW.value   = self.country
    
    # get the list of cities for the country selected and set the 
    # default city selection to be the first city on the list

    self.updateCities()

    # get the list of locations for the city selected and set the 
    # default location selection to be the first location on the list

    self.updateLocations()

    # set up  callback functions for the selection widgets

    self.countrySW.observe(self.countrySWCB,names='value')

    self.citySW.observe(self.citySWCB,names='value')

    self.locationSW.observe(self.locationSWCB,names='value')


  def updateCities(self):
    self.cities         = self.api.cities(country=self.ccode,df=True,limit=1000)
    self.city           = self.cities.iloc[0]['city']
    cities              = list(self.cities['name'])
    self.citySW.options = [i.replace("\n","")for i in  cities]
    self.citySW.value   = self.city

  def updateLocations(self):
  
    self.locations  = self.api.locations(city=self.city,df=True,limit=1000)
    self.location   = self.locations.iloc[0]['location']
    locations       = list(self.locations['location'])
    self.locationSW.options = [i.replace("\n","")for i in  locations]
    self.locationSW.value   = self.location

  def countrySWCB(self,change):
    if change['type'] == 'change':

      # set the country to the new user selected value given 
      # by the "value" attribute in the country selection widget.
      
      self.country    = self.countrySW.value
      self.ccode      = self.countries[self.countries['name'] == self.country]['code'].values[0]

      # get list of cities and set the default city as the first city
      # on the list

      self.updateCities()

      # get list of locations and set the default location as the first location
      # on the list

      self.updateLocations()     

  def citySWCB(self,change):
    if change['type'] == 'change':

      # set the city to the new user selected value given 
      # by the "value" attribute in the city selection widget.

      self.city    = self.citySW.value

      # get list of locations and set the default location as the first location
      # on the list

      self.updateLocations()    

  def locationSWCB(self,change):
    if change['type'] == 'change':

      # set the location to the new user selected value given 
      # by the "value" attribute in the location selection widget.

      self.location    = self.locationSW.value
      #self.showQuery()

  def execQuery(self):
      try:
        df = self.locations.loc[self.locations['location']==self.location].T
        with self.out_cp:
          self.out_cp.clear_output()
          display(df)
      except Exception:
        with self.out_cp:
          self.out_cp.clear_output()
          print('Error getting data')
q = OpenAQGui()
display(q.getCP())

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/openaq/decorators.py:57: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  data = pd.io.json.json_normalize(resp)


In [4]:
from IPython.display import display
from ipywidgets import HBox, VBox, Button,Layout, HTML,Output,GridspecLayout,Dropdown,GridBox,Textarea,Text,Password
import openaq
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


class QueryOpenAq(OpenAQGui):
  def __init__(self,*args,**kwargs):
    OpenAQGui.__init__(self,*args, **kwargs)

  def getCP(self):

    self.title = 'Query OpenAQ'

    self.setLabel()
    
    self.showInfo = Button(description='Query',disabled=False,
                           layout={'width':'auto','border':'3px outset'})
    
    self.showInfo.on_click(self.showQuery)

    self.inpUSR.children+= (VBox([self.showInfo],layout={'overflow':'visible'}),)


    return self.cp

  def showQuery(self,b):
      try:
        df = self.locations.loc[self.locations['location']==self.location].T
        with self.out_cp:
          self.out_cp.clear_output()
          display(df)
      except Exception:
        with self.out_cp:
          self.out_cp.clear_output()
          print('Error getting data')

class PlotOpenAq(OpenAQGui):
  def __init__(self,*args,**kwargs):
    OpenAQGui.__init__(self,*args, **kwargs)

  def getCP(self):

    self.title = 'Query OpenAQ'
    
    self.setLabel()
    
    self.plotStn = Button(description='Plot',disabled=False,
                           layout={'width':'150px','overflow':'visible','border':'3px outset'})
    
    self.plotStn.on_click(self.plotObs)

    self.inpUSR.children+= (VBox([self.plotStn],layout={'overflow':'visible'}),)

    return self.cp
  
  def plotObs(self,b):
    try:
      res = self.api.measurements(location=self.location,parameter='pm25', limit=10000,df=True)
      plot_obs = True
    except Exception:
      print("Error getting data")
      plot_obs = False
      return False
    with self.out_cp:
      plt.ioff()
      fig,ax = plt.subplots(1, figsize=(12, 6))
      sns.set(style="ticks", font_scale=1.35)
      for group, df in res.groupby('location'):
        _df = df.query("value >= 0.0").resample('12h').mean()
        # Convert from ppm to ppb
        _df['value'] *= 1e0
        # Plot the data
        _df.value.plot(ax=ax, label=group)
      ax.legend(loc='best')
      ax.set_ylim(0, None)
      ax.set_ylabel("$PM2.5 \; [\mu m^-3]$", fontsize=18)
      ax.set_xlabel("")
      # move the legend to the side
      plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
      sns.despine(offset=5)
      self.out_cp.clear_output()
      #display(fig)
      plt.show()
      return True
      
from ipywidgets import HBox     
q = PlotOpenAq()
display(q.getCP())

/usr/local/lib/python3.6/dist-packages/openaq/decorators.py:57: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  data = pd.io.json.json_normalize(resp)


In [5]:
from pydap.client import open_url
from pydap.cas.urs import setup_session
import numpy as np
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from ipywidgets import Button,Output

# Explore this link to select the appropriate dataset and variable - https://goldsmr4.gesdisc.eosdis.nasa.gov/opendap/hyrax/MERRA2/. 
# In the example below, I am pulling aerosol fields from MERRA2 diurnal dataset.  When you navigate down to datasets, it has info on
# different variables 

class MerraAero(PanelObject):
  def __init__(self,*args,**kwargs):
    self.title = 'MERRA GUI'
    PanelObject.__init__(self,*args, **kwargs)
    self.opendap_url="https://goldsmr4.gesdisc.eosdis.nasa.gov/opendap/MERRA2_DIURNAL/M2TUNXAER.5.12.4/2019/MERRA2_400.tavgU_2d_aer_Nx.201911.nc4"

    self.plotBW = Button(description='Plot',disabled=False,layout={'width':'auto','border':'3px outset'})
    
    self.plotBW.on_click(self.plotObs)

    self.inpUSR.children+= (VBox([self.plotBW],layout={'overflow':'visible'}),)
  def getCP(self):
    return self.cp
  def plotObs(self,b):
    #Put your NASA Earthdata username and password here
    username = self.pwdDict['NASA Earth Data']['user']
    #username = 'nair@nsstc.uah.edu'
    password = self.pwdDict['NASA Earth Data']['password']
    session = setup_session(username, password, check_url=self.opendap_url)
    dataset = open_url(self.opendap_url, session=session)
    #dataset = open_url(opendap_url)
    lon = dataset['lon'][:]
    lat = dataset['lat'][:]
    aod      = np.squeeze(dataset['TOTEXTTAU'][0,:,:])
    dust_pm  = np.squeeze(dataset['DUSMASS25'][0,:,:])
    salt_pm  = np.squeeze(dataset['SSSMASS25'][0,:,:])
    org_carb = np.squeeze(dataset['OCSMASS'][0,:,:])
    blk_carb = np.squeeze(dataset['BCSMASS'][0,:,:])
    so4      = np.squeeze(dataset['SO4SMASS'][0,:,:])
    pm25 = (1.375*so4 + 1.6*org_carb + blk_carb + dust_pm + salt_pm)*1000000000.0
    eta  = pm25/aod
    lons,lats = np.meshgrid(lon,lat)
    # Set the figure size, projection, and extent
    with self.out_cp:
      plt.ioff()
      self.out_cp.clear_output()
      fig = plt.figure(figsize=(8,4))
      ax = plt.axes(projection=ccrs.Robinson())
      ax.set_global()
      #ax.set_extent([65, 100, 5.0,35.0])
      ax.coastlines(resolution="110m",linewidth=1)
      ax.gridlines(linestyle='--',color='black')
      # Set contour levels, then draw the plot and a colorbar
      clevs = np.arange(230,311,5)
      plt.contourf(lons, lats, pm25,transform=ccrs.PlateCarree(),cmap=plt.cm.jet)
      plt.title('MERRA2 Aerosol Eta Factor, 0 UTC,1 Nov 2019', size=14)
      cb = plt.colorbar(ax=ax, orientation="vertical", pad=0.02, aspect=16, shrink=0.8)
      cb.set_label('K',size=12,rotation=0,labelpad=15)
      cb.ax.tick_params(labelsize=10)
      plt.show()
q = MerraAero()
display(q.getCP())

In [6]:
### Tool to plot tropical cyclone path using Tropycal
### Written by Christopher Phillips
### Direct questions to chris.phillips@uah.edu

### Import required modules
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from datetime import datetime as dt
import ipywidgets as ipw
import matplotlib.pyplot as pp
import numpy
import tropycal.tracks as tracks

### TCPATH class to plot tropical cyclone path
class TCPath(PanelObject):

    ### Constructor method
    def __init__(self,*args,**kwargs):

        #Set title of panel
        self.title = 'TC Path Tool'

        #Derive class from parent object
        PanelObject.__init__(self,*args, **kwargs)

        #Create Cartopy projection
        self.pcp = ccrs.PlateCarree()

        #Create cartopy feature objects
        self.countries = cfeature.NaturalEarthFeature(category='cultural',
            name='admin_0_countries', scale='50m', facecolor='none')
        self.states = cfeature.NaturalEarthFeature(category="cultural",
            name="admin_1_states_provinces_lines", scale="50m", facecolor="none")

        #Create data set parameters
        self.tpc_source = "ibtracs"
        self.available_basins = ["north_atlantic", "south_america", "east_pacific", "west_pacific", "south_pacific", "north_indian", "south_indian", "australian"]

        #Create GUI elements for storm selection
        #Layout for storm selection elements
        menu_layout = {"width":"auto", "height":"auto"}

        #Dropdown list to select hurricane basin
        self.basin_list = ipw.Dropdown(options=self.available_basins, value=None, description="Basin", disabled=False, layout=menu_layout)
        
        #Text box to enter a year
        self.year_box = ipw.BoundedIntText(value=None, min=1851, max=2019, description="Season", disabled=False, layout=menu_layout)

        #Dropdown menu with storm names
        self.storm_list = ipw.Dropdown(options=[None], value=None, description="Storm", disabled=False, layout=menu_layout)

        #Add button to load storm
        self.storm_button = ipw.Button(description="Load Storm", tooltip="Load Storm and Plot Track", disabled=False, layout={"width":"auto", "border":"3px outset", "height":"auto"})

        #Add GUI elements to display list
        dummy = HBox([self.storm_list, self.storm_button], layout={"width":"auto", "height":"auto", "overflow":"visible"})
        self.inpUSR.children+= (VBox([self.basin_list, self.year_box, dummy],layout={"width":"auto", "height":"auto", "overflow":"visible"}),)

        #Add callback function to buttons and menus
        self.storm_button.on_click(self.plot_storm) #Plot the storm track on button click
        self.basin_list.observe(self.load_tropycal) #Load Tropycal when you select a basin
        self.year_box.observe(self.load_season)     #Load storm list when a season is selected

    ### Method to return GUI contorl panel object
    def getCP(self):
        return self.cp

    ### Method to load IBTrACS
    def load_season(self, change):

        #Check if basin menu changed (and not None)
        if ((change["type"] == "change") and (change["name"] == "value") and (self.year_box.value != None)):

            #Load the season
            self.season = self.tpc_dataset.get_season(self.year_box.value)

            #Loop over all storms in season
            names = []
            ids = []
            for k in self.season.dict.keys():
                names.append("{} - {}".format(self.season.dict[k]["name"], self.season.dict[k]["date"][0].strftime("%Y/%m/%d")))
                ids.append(self.season.dict[k]["id"])

            #Update the storm list
            self.storm_list.options = list(zip(names, ids))

        #Returning
        return

    ### Method to load tropycal dataset on basin selection
    def load_tropycal(self, change):
        #Check if basin menu changed (and not None)
        if ((change["type"] == "change") and (change["name"] == "value") and (self.basin_list.value != None)):

            #Load dataset
            self.tpc_dataset = tracks.TrackDataset(self.basin_list.value, self.tpc_source)

        #Returning
        return

    ### Method to plot storm track (button call back)
    def plot_storm(self, button):

        #Load storm
        self.storm = self.tpc_dataset.get_storm(self.storm_list.value)

        #Direct plot output to display
        with self.out_cp:

            #Turn of blocking behavior of pyplot.show
            plt.ioff()

            #Clear any previous output
            self.out_cp.clear_output()
            try:
                pp.close(fig)
            except:
                pass

            #Create figure and axis objects
            fig, ax = pp.subplots(subplot_kw={"projection":self.pcp})
                                
            #Plot storm track
            ax.plot(self.storm.dict["lon"], self.storm.dict["lat"], color="red", transform=self.pcp)

            #Make the plotlook like a map        
            #Set extent (Box around storm track with 10 degree padding
            ax.set_extent([numpy.min(self.storm.dict["lon"])-10, numpy.max(self.storm.dict["lon"])+10, numpy.min(self.storm.dict["lat"])-10, numpy.max(self.storm.dict["lat"])+10])

            #Coasts and countries
            ax.coastlines()
            ax.add_feature(self.states, edgecolor="black", linewidth=1)
            ax.add_feature(self.countries, edgecolor="black", linewidth=1)

            #Gridlines
            gridlines = ax.gridlines(crs=self.pcp, draw_labels=True, linestyle=":", color="black")
            gridlines.xlabels_top = False
            gridlines.ylabels_right = False

            #Map image
            ax.stock_img()

            #Label the plot
            ax.set_title("Storm {}".format(self.storm.dict["name"]), fontsize=14)

            #Display plot
            pp.show()

        #Returning
        return

#Test object
q = TCPath()
display(q.getCP())

In [14]:
### Tool to plot tropical cyclone location in conjunctyion with MERRA
### Written by Christopher Phillips
### Direct questions to chris.phillips@uah.edu

### Import required modules
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from datetime import datetime as dt
import ipywidgets as ipw
import matplotlib.pyplot as pp
import numpy
from pydap.client import open_url
from pydap.cas.urs import setup_session
import tropycal.tracks as tracks

### TCPATH class to plot tropical cyclone path
class TCMERRA(PanelObject):

    ### Constructor method
    def __init__(self,*args,**kwargs):

        #Set title of panel
        self.title = 'TC-MERRA Analysis'

        #Derive class from parent object
        PanelObject.__init__(self,*args, **kwargs)

        #Create Cartopy projection
        self.pcp = ccrs.PlateCarree()

        #Create cartopy feature objects
        self.countries = cfeature.NaturalEarthFeature(category='cultural',
            name='admin_0_countries', scale='50m', facecolor='none')
        self.states = cfeature.NaturalEarthFeature(category="cultural",
            name="admin_1_states_provinces_lines", scale="50m", facecolor="none")

        #Create data set parameters
        self.tpc_source = "ibtracs"
        self.available_basins = ["north_atlantic", "south_america", "east_pacific", "west_pacific", "south_pacific", "north_indian", "south_indian", "australian"]

        #Create GUI elements for storm selection
        #Layout for storm selection elements
        menu_layout = {"width":"auto", "height":"auto"}

        #Dropdown list to select hurricane basin
        self.basin_list = ipw.Dropdown(options=self.available_basins, value=None, description="Basin", disabled=False, layout=menu_layout)
        
        #Text box to enter a year
        self.year_box = ipw.BoundedIntText(value=None, min=1851, max=2019, description="Season", disabled=False, layout=menu_layout)

        #Dropdown menu with storm names
        self.storm_list = ipw.Dropdown(options=[None], value=None, description="Storm", disabled=False, layout=menu_layout)

        #Add button to load storm
        self.storm_button = ipw.Button(description="Load Storm", tooltip="Load Storm and Plot Track", disabled=False, layout={"width":"auto", "border":"3px outset", "height":"auto"})

        #Add dropdown menu with storm dates
        self.date_list = ipw.Dropdown(options=[None], value=None, description="Date", disabled=False, layout=menu_layout)

        #Add GUI elements to display list
        left = VBox([self.basin_list, self.year_box, self.storm_list],layout={"width":"auto", "height":"auto", "overflow":"visible"})
        right = VBox([self.storm_button, self.date_list], layout={"width":"auto", "height":"auto", "overflow":"visible"})
        self.inpUSR.children+= (HBox([left, right],layout={"width":"auto", "height":"auto", "overflow":"visible"}),)

        #Add callback function to buttons and menus
        self.storm_button.on_click(self.load_storm) #Plot the storm track on button click
        self.date_list.observe(self.change_date)
        self.basin_list.observe(self.load_tropycal) #Load Tropycal when you select a basin
        self.year_box.observe(self.load_season)     #Load storm list when a season is selected

    ### Method to change date information for object
    def change_date(self, change):

        #Check for date selection change
        if ((change["type"] == "change") and (change["name"] == "value")):

            #Set current date
            self.date = self.date_list.value

            #Find index of current date in storm info
            for i in range(len(self.storm.dict["date"])):
                if (self.date == self.storm.dict["date"][i]):
                    self.date_ind = i
                    break

            #Replot storm location
            self.plot_storm()

        #Returning
        return

    ### Method to return GUI contorl panel object
    def getCP(self):
        return self.cp

    ### Method to subset data
    ### Generates indices that match desired region of plot
    ### Inputs:
    ###  extent, list of floats, [west lon, east lon, south lat, north lat]
    ###
    ### Outputs,
    ###   subset, tuple of ints (y1, y2, x1, x2), array indices corresponding to subsetted region.
    def get_subset(self):
                                    
        #Calculate coordinates of each bounding side
        dx = self.lons[1]-self.lons[0]
        dy = self.lats[1]-self.lats[0]
        lon1_ind = int((self.storm_extent[0]-self.lons[0])/dx)
        lon2_ind = int((self.storm_extent[1]-self.lons[0])/dx)
        lat1_ind = int((self.storm_extent[2]-self.lats[0])/dy) #Do south first because MERRA stores south first.
        lat2_ind = int((self.storm_extent[3]-self.lats[0])/dy)
        
        self.yind1 = lat1_ind
        self.yind2 = lat2_ind
        self.xind1 = lon1_ind
        self.xind2 = lon2_ind

        #Returning
        return 

    ### Method to load IBTrACS
    def load_season(self, change):

        #Check if basin menu changed (and not None)
        if ((change["type"] == "change") and (change["name"] == "value") and (self.year_box.value != None)):

            #Load the season
            self.season = self.tpc_dataset.get_season(self.year_box.value)

            #Loop over all storms in season
            names = []
            ids = []
            for k in self.season.dict.keys():
                names.append("{} - {}".format(self.season.dict[k]["name"], self.season.dict[k]["date"][0].strftime("%Y/%m/%d")))
                ids.append(self.season.dict[k]["id"])

            #Update the storm list
            self.storm_list.options = list(zip(names, ids))

        #Returning
        return

    ### Method to load data from Tropycal and MERRA
    def load_storm(self, button):

        #Load storm
        self.storm = self.tpc_dataset.get_storm(self.storm_list.value)

        #Update list of dates
        dates = list(d.strftime("%Y/%m/%d-%H00Z") for d in self.storm.dict["date"])
        self.date_list.options=list(zip(dates, self.storm.dict["date"]))
        self.date_ind = 0
        self.date_list.value=self.storm.dict["date"][self.date_ind]

        #Returning
        return

    ### Method to load tropycal dataset on basin selection
    def load_tropycal(self, change):
        
        #Check if basin menu changed (and not None)
        if ((change["type"] == "change") and (change["name"] == "value") and (self.basin_list.value != None)):

            #Load dataset
            self.tpc_dataset = tracks.TrackDataset(self.basin_list.value, self.tpc_source)

        #Returning
        return

    ### Method to plot storm track (button call back)
    def plot_storm(self):

        #Grab extent of storm region
        self.storm_extent = [self.storm.dict["lon"][self.date_ind]-10, self.storm.dict["lon"][self.date_ind]+10, self.storm.dict["lat"][self.date_ind]-10, self.storm.dict["lat"][self.date_ind]+10]

        #Load relevant MERRA data
        #Retrieve username and password
        username = self.pwdDict['NASA Earth Data']['user']
        password = self.pwdDict['NASA Earth Data']['password']

        #Build url
        self.atmos_opendap_url = "https://goldsmr4.gesdisc.eosdis.nasa.gov/opendap/MERRA2/M2T1NXSLV.5.12.4/{}/{:02d}/MERRA2_400.tavg1_2d_slv_Nx.{}.nc4".format(self.date.year, self.date.month, self.date.strftime("%Y%m%d"))

        #Load MERRA
        try:
            session = setup_session(username, password, check_url=self.atmos_opendap_url)
            dataset = open_url(self.atmos_opendap_url, session=session)
        except: #Older data might use a number other than 400 in the url
            self.atmos_opendap_url = "https://goldsmr4.gesdisc.eosdis.nasa.gov/opendap/MERRA2/M2T1NXSLV.5.12.4/{}/{:02d}/MERRA2_300.tavg1_2d_slv_Nx.{}.nc4".format(self.date.year, self.date.month, self.date.strftime("%Y%m%d"))
            session = setup_session(username, password, check_url=self.atmos_opendap_url)
            dataset = open_url(self.atmos_opendap_url, session=session)

        #Pull the correct hour for the analysis
        hour = self.date.hour

        #First pull lat and lons and get subset
        self.lons = numpy.squeeze(dataset["lon"][:])
        self.lats = numpy.squeeze(dataset["lat"][:])
        self.get_subset()

        #Now pull the rest of the analysis data
        uwind = numpy.squeeze(dataset["U10M"][hour, self.yind1:self.yind2, self.xind1:self.xind2])
        vwind = numpy.squeeze(dataset["V10M"][hour, self.yind1:self.yind2, self.xind1:self.xind2])
        pres = numpy.squeeze(dataset["SLP"][hour, self.yind1:self.yind2, self.xind1:self.xind2])/100.0 #Pa -> hPa

        #Direct plot output to display
        with self.out_cp:

            #Turn of blocking behavior of pyplot.show
            plt.ioff()

            #Clear any previous output
            self.out_cp.clear_output()
            try:
                pp.close(fig)
            except:
                pass

            #Create figure and axis objects
            fig, ax = pp.subplots(subplot_kw={"projection":self.pcp})

            #Determine color according to storm intensity
            if (self.storm.dict["wmo_vmax"][self.date_ind] <= 33): #Storm wind is in knots
                c = "blue"
            elif (self.storm.dict["wmo_vmax"][self.date_ind] <= 63):
                c = "green"
            elif (self.storm.dict["wmo_vmax"][self.date_ind] <= 82):
                c = "yellow"
            elif (self.storm.dict["wmo_vmax"][self.date_ind] <= 95):
                c = "orange"
            elif (self.storm.dict["wmo_vmax"][self.date_ind] <= 112):
                c = "red"
            elif (self.storm.dict["wmo_vmax"][self.date_ind] <= 136):
                c = "purple"
            elif (self.storm.dict["wmo_vmax"][self.date_ind] > 136):
                c = "black"
            else: #In case of no wind reports
                c = "white"

            #Add fake lines for legend
            ax.plot(0, 0, "o", color="black")
            ax.plot(0, 0, "o", color="purple")
            ax.plot(0, 0, "o", color="red")
            ax.plot(0, 0, "o", color="orange")
            ax.plot(0, 0, "o", color="yellow")
            ax.plot(0, 0, "o", color="green")
            ax.plot(0, 0, "o", color="blue")

            #Plot MERRA analysis
            cont = ax.contourf(self.lons[self.xind1:self.xind2], self.lats[self.yind1:self.yind2], pres, transform=self.pcp)
            self.cb = pp.colorbar(cont)
            self.cb.set_label("Pressure (hPa)")
            ax.barbs(self.lons[self.xind1:self.xind2][::4], self.lats[self.yind1:self.yind2][::4], uwind[::4,::4], vwind[::4,::4], transform=self.pcp)            

            #Plot storm location (color-coded by intensity)
            ax.scatter(self.storm.dict["lon"][self.date_ind], self.storm.dict["lat"][self.date_ind], color=c, transform=self.pcp)

            #Make the plotlook like a map        
            #Set extent (Box around storm location with 10 degree padding
            ax.set_extent(self.storm_extent)

            #Coasts and countries
            ax.coastlines()
            ax.add_feature(self.states, edgecolor="black", linewidth=1)
            ax.add_feature(self.countries, edgecolor="black", linewidth=1)

            #Gridlines
            gridlines = ax.gridlines(crs=self.pcp, draw_labels=True, linestyle=":", color="black")
            gridlines.xlabels_top = False
            gridlines.ylabels_right = False

            #Map image
            ax.stock_img()

            ax.legend(["Cat 5", "Cat 4", "Cat 3", "Cat 2", "Cat 1", "TS", "TD"], loc="upper right")

            #Label the plot
            ax.set_title("Storm {} - {}".format(self.storm.dict["name"], self.date.strftime("%Y/%m/%d")), fontsize=14)

            #Display plot
            pp.show()

        #Returning
        return

#Test object
q = TCMERRA()
display(q.getCP())

In [16]:
%matplotlib inline
%config InlineBackend.close_figures=False #Don't display plot at end of running code

import functools
from IPython.display import display
from ipywidgets import HBox, VBox, Button,Layout, HTML,Output,GridspecLayout,Dropdown,GridBox,Textarea,Text,Password

class RADashBoard:

  def __init__(self):
    self.out_cp = Output()
    display(self.out_cp)
    self.objDict = {}
    self.objinfoDict = {}
    self.title = 'Research & Applications Dashboard'
    self.nx   = 1
    self.ny   = 1
    self.panelWidth  = [250,500]
    self.panelHeight = [250,500]
    #self.ob  =   OpenAQGui().createCP()
    self.ob  =  QueryOpenAq().getCP()
    self.state = 0
    self.gridGap = 15
    self.gridPad = 15
    self.panelBorder = 3
    self.dbBorder =5
    self.cpView  = 0
    self.pwdDict= {}
    self.createMainCP = True
    self.accList =['NASA Earth Data']
  
  def addAccount(accname):
    self.accList.append(accname)

  def addObject(self,obj,short_name, desc):
    self.objDict[short_name] = obj
    self.objinfoDict[short_name] = desc

  def displayCP(self):
    
    self.rowSW = Dropdown(options=[1,2,3,4],value=1,description='Rows:',
                          disabled=False,layout={'width':'150px'}
                         )
    self.colSW = Dropdown(options=[1,2,3,4],value=1,description='Columns:',
                            disabled=False,layout={'width':'150px'}
                          )
    self.row  = Dropdown(options=[1,2,3,4],value=1,
                         description='# of row:',disabled=True)
    self.col  = Dropdown(options=[1,2,3,4],value=1,
                         description='# of col:',disabled=True)
    self.obj  = Dropdown(options=[],value=None,
                         description='Object:',disabled=True)
    self.conDB = Button(description='Configure Dashboard',disabled=False,
                          layout={'width':'150px','border':'3px outset'}
                         )
    self.topLabel = HTML(disabled=False,layout={'width': 'auto'})
      
    self.topLabel.value = ( '<div style="background-color: #C0C0C0 ;'+ 
                            'padding: 1em 0em 0em 0em;border-bottom-style:groove;border-width:4px">'+
                           '<b><font size="4"><center><h2>'+self.title+'</h2></center></font></b></div>')
    self.rule = HTML(disabled=True,layout=Layout(width='auto',padding='0px'))
      
    self.rule.value = ( '<div >'+'<hr></div>')
    self.cpMain   = VBox([ 
                          HBox([self.topLabel],layout={'justify_content':'center','flex_flow':'column'}),
                          HBox([
                                HBox([self.rowSW],layout={'justify_content':'flex-start'}),
                                HBox([self.colSW],layout={'justify_content':'flex-end'})
                               ],layout={'padding':'15px'}
                              ),
                          HBox([self.conDB],layout={'justify_content':'center','padding':'15px'})
                               ],layout={'border':'3px groove','width': '400px'}
                        )
    self.rowSW.observe(self.rowSWCB,names='value')
    self.colSW.observe(self.colSWCB,names='value')
    self.conDB.on_click(self.configDBCB)
    with self.out_cp:
      display(self.cpMain)

  def rowSWCB(self,change):
    if change['type'] == 'change':

      # set the city to the new user selected value given 
      # by the "value" attribute in the city selection widget.

      self.ny    = int(self.rowSW.value)
      

  def colSWCB(self,change):
    if change['type'] == 'change':

      # set the city to the new user selected value given 
      # by the "value" attribute in the city selection widget.

      self.nx    = int(self.colSW.value)
      

  def rowSWCB(self,change):
    if change['type'] == 'change':

      # set the city to the new user selected value given 
      # by the "value" attribute in the city selection widget.

      self.ny    = int(self.rowSW.value)
      

  def colSWCB(self,change):
    if change['type'] == 'change':

      # set the city to the new user selected value given 
      # by the "value" attribute in the city selection widget.

      self.nx    = int(self.colSW.value)
      


  def configDBCB(self,b):

    self.out_cp.clear_output()

    self.objList = [*(self.objDict)]

    pw = self.panelWidth[0]

    lw = (pw*self.nx)+((self.nx-1)*self.gridGap)+(self.gridPad*2)

    gw = lw + (2*self.dbBorder)

    lw1 = lw-2

    self.topLabel.layout = {'width': str(lw-2)+'px'}

    gap = str(self.gridGap)+'px'

    pd = str(self.gridPad)

    pad = pd+'px '+pd+'px '+pd+'px '+pd+'px'

    self.gridDB1 = GridspecLayout(self.ny,self.nx,
                                  layout={
                                          'scroll':'False',
                                          'grid_gap':gap,
                                          'padding':pad
                                          }
                                  )
    self.objSW = [ ([0] * self.nx) for y in range(self.ny) ]
    self.objinfoTW = [ ([0] * self.nx) for y in range(self.ny) ]
    self.pwTW = [ ([0] * self.nx) for y in range(self.ny) ]
    self.phTW = [ ([0] * self.nx) for y in range(self.ny) ]

    txw = str(int(0.96*float(self.panelWidth[0])))+'px'
    pw = str(self.panelWidth[0])+'px'
    ph = str(self.panelHeight[0])+'px'
    pb = str(self.panelBorder)+'px groove'

    for i in range(0,self.ny):
      for j in range(0,self.nx):
        desc = 'Panel('+str(i+1)+','+str(j+1)+'):'
        self.objSW[i][j]  = Dropdown(options=self.objList,value=self.objList[0],
                                    description=desc,disabled=False,
                                    layout={'width':txw})
        objinfo = self.objinfoDict[self.objList[0]]
        self.objinfoTW[i][j] = Textarea(value=objinfo,placeholder='',description='',
                                    disabled=False,layout={'width':txw,'border':'2px inset'})
        self.pwTW[i][j] = Text(value=str(self.panelWidth[1]),placeholder='',description='Panel Width:',
                                    disabled=False,layout={'width':txw,'border':'2px inset'})
        self.phTW[i][j] = Text(value=str(self.panelHeight[1]),placeholder='',description='Panel Height',
                                    disabled=False,layout={'width':txw,'border':'2px inset'})
        
        self.gridDB1[i,j] = VBox([self.objSW[i][j],self.pwTW[i][j],self.phTW[i][j],self.objinfoTW[i][j]],layout={'border':'2px solid black'})

        self.objSW[i][j].observe(functools.partial(self.objSWCB, irow_=i, jcol_=j),names='value')
        self.phTW[i][j].observe(functools.partial(self.phTWCB, irow_=i, jcol_=j),names='value')

    gp  = str(self.gridPad)+'px'
    dbb = str(self.dbBorder)+'px groove'
    dbw = str(gw)+'px'

    self.pmLabel = HTML(disabled=False,layout={'width': 'auto','flex_flow':'column'})
    
    self.pmLabel.value = ( '<div style="background-color: #C0C0C0 ;'+ 
                           'border-top-style:groove;border-width:3px'
                           'padding: 1em 0em 0em 0em;border-bottom-style:groove;border-width:3px">'+
                           '<b><font size="4"><center><h3>Password Manager</h3></center></font></b></div>')
    self.accSW = Dropdown(options=self.accList,value=self.accList[0],
                                    description='Account:',disabled=False,
                                    layout={'width':txw})
    self.usrTW = Text(value='',placeholder='',description='Username:',disabled=False,
                                    layout={'width':txw})
    
    self.pwdPW = Password(value='',placeholder='',description='Password:',disabled=False,
                                    layout={'width':txw})
    self.addPWD = Button(description='Add Account',disabled=False,
                        layout={'width':'150px','border':'3px outset'}
                        )
    self.createDB = Button(description='Create Dashboard',disabled=False,
                        layout={'width':'150px','border':'3px outset'}
                        )
    self.addPWD.on_click(self.addPWDCB)
    self.createDB.on_click(self.createDBCB)
    self.reconfigDB = Button(description='Reconfigure Dashboard',disabled=False,
                        layout={'width':'180px','border':'3px outset'}
                        )
    self.reconfigDB.on_click(self.reconfigDBCB)
    
    self.cp   = VBox([ 
                       HBox([self.topLabel],layout={'flex_flow':'column'}),
                       HBox([self.gridDB1]),
                       HBox([self.pmLabel],layout={'flex_flow':'column'}),
                       VBox([self.accSW,self.usrTW,self.pwdPW]),
                       HBox([self.addPWD],layout={'justify_content':'center'}),
                       self.rule,
                       HBox([self.reconfigDB,self.createDB],layout={'justify_content':'center','padding':gp})
                     ],layout={'border':dbb,'width':dbw}
                    )
    with self.out_cp:
      self.out_cp.clear_output()
      display(self.cp)
  def objSWCB(self,change,irow_,jcol_):
    self.objinfoTW[irow_][jcol_].value =self.objinfoDict[self.objSW[irow_][jcol_].value]

  def phTWCB(self,change,irow_,jcol_):
    self.objinfoTW[irow_][jcol_].value =('Hint: Set the same height of all the panels in'+ 
                                         ' a row for optimal panel layout')
    

  def addPWDCB(self,b):
    self.pwdDict[self.accSW.value]={'user':self.usrTW.value,'password':self.pwdPW.value}
    

  def createDBCB(self,b):
    self.out_cp.clear_output()
    wd = [0]*self.ny
    for i in range(self.ny):
      for j in range(self.nx):
        wd[i] += int(self.pwTW[i][j].value)

    tpw = max(wd)
    lw = tpw+((self.nx-1)*self.gridGap)+(self.gridPad*2)
    self.topLabel.layout = {'width': 'auto'}
    gw = lw + (2*self.dbBorder)
    gap = str(self.gridGap)+'px'
    pd = str(self.gridPad)
    pad = pd+'px '+pd+'px '+pd+'px '+pd+'px'
    self.gridDB2  = GridspecLayout(self.ny,self.nx,
                                  layout={
                                          'scroll':'True',
                                          'grid_gap':gap,
                                          'padding':pad
                                          }
                                  )
    ph = str(self.panelHeight[1])+'px'
    pb = str(self.panelBorder)+'px groove'

    for i in range(0,self.ny):
      for j in range(0,self.nx):
        pw = self.pwTW[i][j].value+'px'
        ph = self.phTW[i][j].value+'px'
        obj = self.objDict[self.objSW[i][j].value]()
        obj.pwdDict = self.pwdDict
        obj.spacer.layout={'width':pw}
        obj = obj.getCP()
        obj.layout={'overflow_x':'visible','overflow_y':'visible'}
        self.gridDB2[i,j] = HBox([obj],layout={'height': ph,'width': pw,'border':pb})
    gp  = str(self.gridPad)+'px'
    dbb = str(self.dbBorder)+'px groove'
    dbw = str(gw)+'px'

    self.cp   = VBox([ VBox([self.topLabel,self.gridDB2],layout={'flex_flow':'column'}),
                       HBox([self.reconfigDB],layout={'justify_content':'center','padding':gp})
                     ],layout={'border':dbb,'width':dbw}
                    )
    with self.out_cp:
      self.out_cp.clear_output()
      display(self.cp)
  def reconfigDBCB(self,b):
    with self.out_cp:
      self.out_cp.clear_output()
      self.topLabel.layout={'flex_flow':'column'}
      display(self.cpMain)

db = RADashBoard()
db.title = 'Air Pollution Research & Applications Dashboard'
db.addObject(QueryOpenAq,'OpenAQ Query','Tool for querying the OpenAQ database. This is a lot of fun  for all.')
db.addObject(PlotOpenAq,'Plot OpenAQ','Tool for plotting time series of OpenAQ stations')
db.addObject(MerraAero,'Plot Merra','Tool for plotting Merra Aerosol Spatial Plots')
db.addObject(TCPath, "Plot TC Track", "Tool for plotting a tropical cyclone track")
db.addObject(TCMERRA, "Plot TC with MERRA", "Tool for plotting TC location on top of MERRA surface analysis")
db.displayCP()

Output()

/usr/local/lib/python3.6/dist-packages/openaq/decorators.py:57: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  data = pd.io.json.json_normalize(resp)


--> Starting to read in ibtracs data
--> Completed reading in ibtracs data (8.35 seconds)
